In [1]:
#In this script we will create star schema tables (fact table and dimensional tables) of the dataset
#we will save the tables data to csv files

In [103]:
#We will concatenate the 3 slices of the cleaned data
import pandas as pd
data1=pd.read_csv('Amazon-Sale-Report-0.csv')
data2=pd.read_csv('Amazon-Sale-Report-1.csv')
data3=pd.read_csv('Amazon-Sale-Report-2.csv')
cleaned_data=pd.concat([data1,data2,data3])
cleaned_data.shape

(122073, 22)

In [139]:
# ============================================================
# FULL STAR SCHEMA BUILD (dims + fact) - IDEMPOTENT VERSION
# - Safe to run multiple times
# - Fixes *_x/*_y merge collisions
# ============================================================

# 0) Start from a clean copy of the ORIGINAL cleaned_data
df = cleaned_data.copy()

# If you previously ran other scripts, remove any old keys to avoid collisions
for col in ["product_id", "date_id", "fulfilment_id", "loc_id", "location_id"]:
    if col in df.columns:
        df = df.drop(columns=[col])

# 1) Parse Date
df["Date"] = pd.to_datetime(df["Date"], errors="coerce").dt.normalize()

# 2) Robust cleanup for location columns
loc_cols = ["ship-postal-code", "ship-state", "ship-city"]
for c in loc_cols:
    df[c] = df[c].astype(str).str.strip().str.upper()

df[loc_cols] = df[loc_cols].replace({"NAN": pd.NA, "NONE": pd.NA, "NULL": pd.NA, "": pd.NA})
df["ship-postal-code"] = df["ship-postal-code"].str.replace(r"\.0$", "", regex=True)

# 3) Normalize SKU (for product mapping)
df["SKU"] = df["SKU"].astype(str).str.strip().str.upper()
df["SKU"] = df["SKU"].replace({"NAN": pd.NA, "NONE": pd.NA, "NULL": pd.NA, "": pd.NA})

# 4) Normalize fulfilment attributes
ful_cols = ["Fulfilment", "Sales_Channel", "ship-service-level", "Courier_Status", "B2B"]
for c in ful_cols:
    df[c] = df[c].astype(str).str.strip().str.upper()

df[ful_cols] = df[ful_cols].replace({"NAN": pd.NA, "NONE": pd.NA, "NULL": pd.NA, "": pd.NA})

# ============================================================
# DIMENSIONS
# ============================================================

# dim_product (grain = SKU)
dim_product = (
    df[["SKU", "Style", "Category", "Size", "ASIN"]]
    .dropna(subset=["SKU"])
    .drop_duplicates(subset=["SKU"])
    .reset_index(drop=True)
)
dim_product["product_id"] = range(1, len(dim_product) + 1)
dim_product = dim_product[["product_id", "SKU", "Style", "Category", "Size", "ASIN"]]

# dim_location (grain = postal+state+city)
dim_location = (
    df[["ship-postal-code", "ship-state", "ship-city"]]
    .dropna(subset=loc_cols)
    .drop_duplicates(subset=loc_cols)
    .reset_index(drop=True)
)
dim_location["loc_id"] = range(1, len(dim_location) + 1)
dim_location = dim_location[["loc_id", "ship-state", "ship-city", "ship-postal-code"]]

# Add Unknown location row (so FK is never null)
unknown_loc = pd.DataFrame([{
    "loc_id": 0,
    "ship-state": "UNKNOWN",
    "ship-city": "UNKNOWN",
    "ship-postal-code": "UNKNOWN"
}])
dim_location = pd.concat([unknown_loc, dim_location], ignore_index=True)

# dim_date (grain = Date)
dim_date = (
    df[["Date"]]
    .dropna()
    .drop_duplicates()
    .sort_values("Date")
    .reset_index(drop=True)
)
dim_date["date_id"] = range(1, len(dim_date) + 1)
dim_date["day"] = dim_date["Date"].dt.day
dim_date["month"] = dim_date["Date"].dt.month
dim_date["year"] = dim_date["Date"].dt.year
dim_date = dim_date[["date_id", "Date", "day", "month", "year"]]

# dim_fulfilment (grain = unique fulfilment combo)
dim_fulfilment = (
    df[ful_cols]
    .drop_duplicates()
    .reset_index(drop=True)
)
dim_fulfilment["fulfilment_id"] = range(1, len(dim_fulfilment) + 1)
dim_fulfilment = dim_fulfilment[["fulfilment_id"] + ful_cols]

# ============================================================
# MAP KEYS BACK INTO df (merges)
# ============================================================

# product_id
df = df.merge(dim_product[["SKU", "product_id"]], on="SKU", how="left")

# date_id
df = df.merge(dim_date[["Date", "date_id"]], on="Date", how="left")

# fulfilment_id
df = df.merge(dim_fulfilment[ful_cols + ["fulfilment_id"]], on=ful_cols, how="left")

# loc_id (merge by full location keys)
df = df.merge(
    dim_location[["ship-postal-code", "ship-state", "ship-city", "loc_id"]],
    on=["ship-postal-code", "ship-state", "ship-city"],
    how="left"
)
df["loc_id"] = df["loc_id"].fillna(0).astype(int)

# ============================================================
# FACT TABLE
# ============================================================
fact_orders = (
    df[["Order_ID", "date_id", "product_id", "loc_id", "fulfilment_id", "Qty", "Amount"]]
    .rename(columns={
        "Order_ID": "order_id",
        "loc_id": "location_id",
        "Qty": "qty",
        "Amount": "amount"
    })
    .reset_index(drop=True)
)

# ============================================================
# VALIDATION
# ============================================================
missing_fk = fact_orders[["date_id", "product_id", "location_id", "fulfilment_id"]].isna().sum()
print("Missing foreign keys per column:\n", missing_fk)

unknown_count = (fact_orders["location_id"] == 0).sum()
print(f"Rows assigned to UNKNOWN location_id=0: {unknown_count}")

# Now you have: dim_product, dim_location, dim_date, dim_fulfilment, fact_orders


Missing foreign keys per column:
 date_id          0
product_id       0
location_id      0
fulfilment_id    0
dtype: int64
Rows assigned to UNKNOWN location_id=0: 0


In [141]:
#Saving tables to csv files
dim_product.to_csv("dim_product.csv", index=False)
dim_location.to_csv("dim_location.csv", index=False)
dim_date.to_csv("dim_date.csv", index=False)
dim_fulfilment.to_csv("dim_fulfilment.csv", index=False)

# ---- Fact table ----
fact_orders.to_csv("fact_orders.csv", index=False)